In [1]:
import xlrd
from pathlib import Path
import pandas as pd
import numpy as np
from numba import decorators
import librosa

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
# read in the data file
# Give the location of the file 

df = pd.read_excel(r'data/data.xlsx', sheet_name='totals')
# print(df)

In [4]:
## LOADING IN DATASETS

dataset = Path.cwd().joinpath("SongEmotionDataset")
datasheet = Path.cwd().joinpath("data") # for csua

#emotion labels
label_loc = datasheet.joinpath("data.xlsx")
wb = xlrd.open_workbook(label_loc) 
sheet = wb.sheet_by_index(1)

#emotion arr
emotions = ["amazement", "solemnity", "tenderness", "nostalgia", "calmness", "power", "joyful activation", "tension", "sadness"]

train_song = []
test_song = []
train_emotion = []
test_emotion = []

for i in range(1, 401):
    count_total = sheet.cell_value(i, 11)
    if i % 5 == 0:
        test_song.append(dataset.joinpath("{}.mp3".format(i)))
        emotion_arr = []
        for j in range(5):
            emotion_arr.append(sheet.cell_value(i, 2 + j))
        test_emotion.append(torch.tensor(emotion_arr, device=device).float())
    else:
        train_song.append(dataset.joinpath("{}.mp3".format(i)))
        emotion_arr = []
        for j in range(9):
            emotion_arr.append(sheet.cell_value(i, 2 + j))
        train_emotion.append(torch.tensor(emotion_arr, device=device))

print(len(train_song), len(test_song))
print(len(train_emotion), len(test_emotion))

320 80
320 80


In [5]:
train_emotion

[tensor([ 7., 16., 10., 14., 30.,  1.,  4.,  3., 15.], device='cuda:0'),
 tensor([ 5.,  8., 20., 16., 35.,  2.,  1.,  1.,  5.], device='cuda:0'),
 tensor([ 8., 16.,  6.,  8.,  3., 13., 19., 11.,  7.], device='cuda:0'),
 tensor([ 5.,  6., 22., 19., 32.,  1.,  0.,  1.,  6.], device='cuda:0'),
 tensor([ 5., 10., 10., 14., 26.,  3., 11.,  6.,  6.], device='cuda:0'),
 tensor([16.,  4.,  7.,  4.,  5., 10., 38.,  7.,  1.], device='cuda:0'),
 tensor([ 4., 13., 15., 14., 22.,  2.,  9.,  7.,  7.], device='cuda:0'),
 tensor([ 5., 12., 14., 18., 21.,  2.,  3.,  1., 16.], device='cuda:0'),
 tensor([ 2., 14., 11., 18., 17.,  2.,  3.,  6., 18.], device='cuda:0'),
 tensor([16.,  3.,  8.,  8.,  8.,  4., 38.,  4.,  2.], device='cuda:0'),
 tensor([ 1., 15.,  7., 19., 18.,  7.,  1., 11., 12.], device='cuda:0'),
 tensor([ 6., 10., 15., 16., 19.,  4.,  7.,  8.,  6.], device='cuda:0'),
 tensor([ 4., 12.,  2.,  2.,  1., 31.,  5., 28.,  6.], device='cuda:0'),
 tensor([ 4.,  7., 25., 19., 28.,  0.,  0.,  2.,  8

In [6]:
class SongEmotionDataset(Dataset):
    """
    Song Emotion Dataset. Uses librosa to process mp3 files.
    Takes first 20 seconds, and samples every 10 to get processed audio tensor.
    """

    def __init__(self, mp3, labels, transform=None):
        """
        Args:
            mp3: list of paths to mp3 files
            labels: list of labels
        """
        self.labels = labels
        self.mp3 = mp3
        
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        data, rate = librosa.load(self.mp3[index], sr=16000, duration=20)
        assert rate == 16000
        sample_tensor = torch.tensor(data, device=device).float()
        downsampled_tensor = sample_tensor[::10]
        
        return downsampled_tensor, F.softmax(self.labels[index])

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(1, 128, 80, 4)
        self.bn1 = nn.BatchNorm1d(128)
        self.pool1 = nn.MaxPool1d(4)
        self.conv2 = nn.Conv1d(128, 128, 3)
        self.bn2 = nn.BatchNorm1d(128)
        self.pool2 = nn.MaxPool1d(4)
        self.conv3 = nn.Conv1d(128, 256, 3)
        self.bn3 = nn.BatchNorm1d(256)
        self.pool3 = nn.MaxPool1d(4)
        self.conv4 = nn.Conv1d(256, 512, 3)
        self.bn4 = nn.BatchNorm1d(512)
        self.pool4 = nn.MaxPool1d(4)
        self.avgPool = nn.AvgPool1d(30) #input should be 512x30 so this outputs a 512x1
        self.fc1 = nn.Linear(512, 9)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(self.bn1(x))
        x = self.pool1(x)
        x = self.conv2(x)
        x = F.relu(self.bn2(x))
        x = self.pool2(x)
        x = self.conv3(x)
        x = F.relu(self.bn3(x))
        x = self.pool3(x)
        x = self.conv4(x)
        x = F.relu(self.bn4(x))
        x = self.pool4(x)
        x = self.avgPool(x)
        x = x.permute(0, 2, 1) #change the 512x1 to 1x512
        x = self.fc1(x)
        return F.log_softmax(x, dim = 2)

model = Net()
model.to(device)
print(model)

Net(
  (conv1): Conv1d(1, 128, kernel_size=(80,), stride=(4,))
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(128, 128, kernel_size=(3,), stride=(1,))
  (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(128, 256, kernel_size=(3,), stride=(1,))
  (bn3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool3): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv1d(256, 512, kernel_size=(3,), stride=(1,))
  (bn4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool4): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (avgPool): AvgPool1d(kernel_size=(30,), stride=(30,), 

In [8]:
train_set = SongEmotionDataset(train_song, train_emotion)
test_set = SongEmotionDataset(test_song, test_emotion)
print("Train set size: " + str(len(train_set)))
print("Test set size: " + str(len(test_set)))

kwargs = {'num_workers': 1, 'pin_memory': True} if device == 'cuda' else {} #needed for using datasets on gpu
train_loader = torch.utils.data.DataLoader(train_set, batch_size = 8, shuffle = True, **kwargs)
test_loader = torch.utils.data.DataLoader(test_set, batch_size = 8, shuffle = True, **kwargs)

optimizer = optim.Adam(model.parameters(), lr = 0.01, weight_decay = 0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 20, gamma = 0.1)

Train set size: 320
Test set size: 80


In [9]:
def train(model, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        data.unsqueeze_(1)
        data = data.requires_grad_() #set requires_grad to True for training
        output = model(data)
        output = output.view(-1, len(emotions))
#         print(output.shape, target.shape)
#         print(output, target)
        loss = F.kl_div(output, target)
        loss.backward()
        optimizer.step()
#         scheduler.step()
        if batch_idx % log_interval == 0: #print training stats
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss))

In [10]:
def test(model, epoch):
    model.eval()
    correct = 0
    for data, target in test_loader:
        data.unsqueeze_(1)
        output = model(data)
        output = output.permute(1, 0, 2)
        pred = output.max(2)[1] # get the index of the max log-probability
        correct += pred.eq(target.max(1)[1]).cpu().sum().item()
    print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
import warnings

log_interval = 5
warnings.filterwarnings("ignore")
for epoch in range(1, 41):
    print("training epoch " + str(epoch))
    if epoch == 31:
        print("First round of training complete. Setting learn rate to 0.001.")
#     scheduler.step()
    train(model, epoch)
    scheduler.step()
    test(model, epoch)

training epoch 1
Train Epoch: 1 [0/320 (0%)]	Loss: 0.238086
Train Epoch: 1 [40/320 (12%)]	Loss: 0.543195
Train Epoch: 1 [80/320 (25%)]	Loss: 0.277862
Train Epoch: 1 [120/320 (38%)]	Loss: 0.275000
Train Epoch: 1 [160/320 (50%)]	Loss: 0.128894
Train Epoch: 1 [200/320 (62%)]	Loss: 0.191396
Train Epoch: 1 [240/320 (75%)]	Loss: 0.220283
Train Epoch: 1 [280/320 (88%)]	Loss: 0.195243

Test set: Accuracy: 21/80 (26%)

training epoch 2
Train Epoch: 2 [0/320 (0%)]	Loss: 0.170415
Train Epoch: 2 [40/320 (12%)]	Loss: 0.197089
Train Epoch: 2 [80/320 (25%)]	Loss: 0.203403
Train Epoch: 2 [120/320 (38%)]	Loss: 0.202500
Train Epoch: 2 [160/320 (50%)]	Loss: 0.189248
Train Epoch: 2 [200/320 (62%)]	Loss: 0.190571
Train Epoch: 2 [240/320 (75%)]	Loss: 0.205890
Train Epoch: 2 [280/320 (88%)]	Loss: 0.166318

Test set: Accuracy: 27/80 (34%)

training epoch 3
Train Epoch: 3 [0/320 (0%)]	Loss: 0.150036
Train Epoch: 3 [40/320 (12%)]	Loss: 0.133074
Train Epoch: 3 [80/320 (25%)]	Loss: 0.141485
Train Epoch: 3 [120/32

Train Epoch: 20 [200/320 (62%)]	Loss: 0.127817
Train Epoch: 20 [240/320 (75%)]	Loss: 0.105206
Train Epoch: 20 [280/320 (88%)]	Loss: 0.118119

Test set: Accuracy: 24/80 (30%)

training epoch 21
Train Epoch: 21 [0/320 (0%)]	Loss: 0.120037
Train Epoch: 21 [40/320 (12%)]	Loss: 0.187839
Train Epoch: 21 [80/320 (25%)]	Loss: 0.096697
Train Epoch: 21 [120/320 (38%)]	Loss: 0.168122
Train Epoch: 21 [160/320 (50%)]	Loss: 0.106986
Train Epoch: 21 [200/320 (62%)]	Loss: 0.132516
Train Epoch: 21 [240/320 (75%)]	Loss: 0.094457
Train Epoch: 21 [280/320 (88%)]	Loss: 0.115992

Test set: Accuracy: 19/80 (24%)

training epoch 22
Train Epoch: 22 [0/320 (0%)]	Loss: 0.114777
Train Epoch: 22 [40/320 (12%)]	Loss: 0.126823
Train Epoch: 22 [80/320 (25%)]	Loss: 0.130811
Train Epoch: 22 [120/320 (38%)]	Loss: 0.103881
Train Epoch: 22 [160/320 (50%)]	Loss: 0.136740
Train Epoch: 22 [200/320 (62%)]	Loss: 0.145961
Train Epoch: 22 [240/320 (75%)]	Loss: 0.172568
Train Epoch: 22 [280/320 (88%)]	Loss: 0.130506

Test set: Ac

## NOTES

below is the mfccs notes / random code

In [ ]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())
# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])
torch.save(model.state_dict(), 'dataset_model_soundemotion.pt')

In [8]:
# audio, sample_rate = librosa.load("SongEmotionDataset/1.mp3", res_type='kaiser_fast')
# # [print(x) for x in audio]

# #convert audio into 2d array
# mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
# # mfccsscaled = np.mean(mfccs.T,axis=0)
# print(mfccs.shape, audio.shape)
# mfccs

In [10]:
# audio_tensor = torch.tensor(audio)
# audio_tensor
# audio_tensor.shape

In [16]:
# for sound_file in data_path.iterdir():
#     if ".mp3" in str(sound_file):
#         print(sound_file)
#         audio, sample_rate = librosa.load(str(sound_file), res_type='kaiser_fast')
        
    